Import data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fedesoriano/heart-failure-prediction")

print("Path to dataset files:", path)

Preprocessing

Exploratory Data Analysis 

In [ ]:
Model evaluation